# Developing Neuroglancer Tool

In [1]:
from PIL import Image
from PIL import ImageFile
import numpy as np
import os
import neuroglancer
import numpy as np
import imageio
import h5py
from cloudvolume import CloudVolume
from tqdm import tqdm
import gc
import webbrowser
import tempfile
from scipy.ndimage import label, generate_binary_structure, find_objects, center_of_mass
from scipy.spatial.distance import euclidean
import cv2

In [2]:
# Directories and output files
raw_image_dir = '/media/mitochondria/Elements/spineheads/raw_images'
segmentation_dir = '/media/mitochondria/Elements/spineheads/segmentations'
raw_output_file = './dataset/raw_images_h5/all_raw_images.h5'
seg_output_file = './dataset/seg_images_h5/all_seg_images.h5'
seg_24bit_output_file = './dataset/seg_images_h5/all_seg_24_images.h5'
n = 5

In [3]:
# Helper functions
def choose_first_n_images(file_path, dataset_name, n, array_dtype = None):
    with h5py.File(file_path, 'r') as fl:
        tmp_dataset = fl[dataset_name]
        images = []
        for i in range(n):
            images.append(tmp_dataset[i])
        if array_dtype is None:
            return np.array(images)
        return np.array(images, dtype = array_dtype)

In [4]:
ip = 'localhost'
port = 9999
neuroglancer.set_server_bind_address(bind_address=ip, bind_port=port)
viewer = neuroglancer.Viewer()

res = neuroglancer.CoordinateSpace(
    names=['z', 'y', 'x'],
    units=['nm', 'nm', 'nm'],
    scales=[60, 4, 4]
)

print('Load raw image and segmentation.')
im = choose_first_n_images(raw_output_file, 'raw_images', n)
gt = choose_first_n_images(seg_output_file, 'seg_images', n, array_dtype=np.uint32)
gt_copy = gt.copy()

print(im.shape, gt.shape)

def ngLayer(data, res, oo=[0, 0, 0], tt='segmentation'):
    return neuroglancer.LocalVolume(data, dimensions=res, volume_type=tt, voxel_offset=oo)
im_layer = ngLayer(im, res, tt='image')
gt_layer = ngLayer(gt, res, tt='segmentation')
with viewer.txn() as s:
    s.layers.append(name='im', layer=im_layer)
    s.layers.append(name='gt', layer=gt_layer)


print(viewer)

Load raw image and segmentation.
(5, 15260, 15217) (5, 15260, 15217)
http://localhost:9999/v/9795f4ba1fdfd731c11492e0f2df89255f646323/


In [5]:
# # - Test Visualizing RGB Segmentation
# ip = 'localhost'
# port = 8888
# neuroglancer.set_server_bind_address(bind_address=ip, bind_port=port)
# viewer_rgb = neuroglancer.Viewer()
# # assume a viewer with is already created

# # coordinate space for gray-scale volume (z,y,x)
# res0 = neuroglancer.CoordinateSpace(
#         names=['z', 'y', 'x'],
#         units=['nm', 'nm', 'nm'],
#         scales=[60, 4, 4])

# # coordinate space for RGB volume (c,z,y,x)
# res1 = neuroglancer.CoordinateSpace(
#         names=['c^', 'z', 'y', 'x'],
#         units=['', 'nm', 'nm', 'nm'],
#         scales=[3, 60, 4, 4])

# im_rgb = choose_first_n_images(seg_24bit_output_file, 'seg_images', n, array_dtype = np.uint32)
# im_rgb = np.transpose(im_rgb.copy(), (3, 0, 1, 2))


# def ngLayer(data,res,oo=[0,0,0],tt='segmentation'):
#     return neuroglancer.LocalVolume(data,dimensions=res,volume_type=tt,voxel_offset=oo)

# with viewer_rgb.txn() as s:
#     # im: 3d array in (z,y,x). im_rgb: 4d array in (c,z,y,x), c=3
#     s.layers.append(name='im',layer=ngLayer(im,res0,tt='image')),
#     # s.layers.append(name='im_rgb',layer=ngLayer(im_rgb,res1,oo=[0,0,0,0],tt='image'),
#     # shader="""
#     #     void main() {
#     #     emitRGB(vec3(toNormalized(getDataValue(0)),
#     #     toNormalized(getDataValue(1)),
#     #     toNormalized(getDataValue(2))));
#     #     }
#     # """
#     # )
#     s.layers.append(name='im_rgb',layer=ngLayer(im_rgb,res1,oo=[0,0,0,0],tt='segmentation'))
# print(viewer_rgb)

## Develop basic functions

In [6]:
# # - show mouse voxel and selected layer value
# num_actions = 0
# def my_action(s):
#     global num_actions
#     num_actions += 1
#     with viewer.config_state.txn() as st:
#       st.status_messages['mouse_info'] = ('Got action %d: mouse position = %r selected value = %s' %
#                                      (num_actions, s.mouse_voxel_coordinates, s.selected_values))
# viewer.actions.add('mouse_coordinate', my_action)
# with viewer.config_state.txn() as s:
#     s.input_event_bindings.viewer['keyt'] = 'mouse_coordinate'
#     s.status_messages['mouse_info'] = 'Mouse Info'

In [7]:
# # - mousedown is problematic
# def add_annotation(s):

#     with viewer.config_state.txn() as st:
#         st.status_messages['annotation info'] = ('Got action: mouse position = %r selected value = %s' %
#                                                  (s.mouse_voxel_coordinates, s.selected_values))
# viewer.actions.add("add-annotation", add_annotation)
# with viewer.config_state.txn() as s:
#     s.input_event_bindings.viewer['shift+keyq'] = "add-annotation"
#     s.status_messages['annotation info'] = ('start to add annotations')

In [8]:
# # - Center Function - how check all available keys
# def center_image(sts):
#     with viewer.txn() as s:
#         s.voxel_coordinates = [0, gt.shape[1]/2, gt.shape[2]/2]
# viewer.actions.add('center_image', center_image)
# with viewer.config_state.txn() as st:
#     st.input_event_bindings.viewer['shift+keyc'] = 'center_image'

In [9]:
# # - Generate All Segmentation
# all_unique_ids = np.unique(gt)
# def generate_all_meshes(st):
#     with viewer.config_state.txn() as st:
#         st.status_messages['generate_meshes'] = (f'Start generating all meshes')
#     with viewer.txn() as s:
#         for i in all_unique_ids:
#             s.layers['segmentation'].segments.add(i)
#     with viewer.config_state.txn() as st:
#         st.status_messages['generate_meshes'] = (f'Finish generating all meshes')
# viewer.actions.add('generate_all_meshes', generate_all_meshes)
# with viewer.config_state.txn() as st:
#     st.input_event_bindings.viewer['control+shift+keyg'] = 'generate_all_meshes'

In [10]:
# # - Screenshot 3D meshes
# from ipywidgets import Image
# count = 0
# screenshot_dir = './screenshot'
# def screenshot_3D_meshes(st):
#     global count
#     global screenshot_dir
#     if not os.path.exists(screenshot_dir):
#         os.makedirs(screenshot_dir)
#     with viewer.txn() as s:
#         s.layout = '3d'
#         s.projection_scale = 5000
#     screenshot = viewer.screenshot(size=[1000, 1000])
#     screenshot_image = Image(value=screenshot.screenshot.image)
#     screenshot_image.save(screenshot_dir + f'/screenshot_{count:03d}.png')
#     count += 1
# viewer.actions.add('screenshot_3D_meshes', screenshot_3D_meshes)
# with viewer.config_state.txn() as st:
#     st.input_event_bindings.viewer['shift+keyf'] = 'screenshot_3D_meshes'

### Develop Merge and Cut Functions

In [11]:
class merge_split_function:
    def __init__(self, viewer, gt, res, gt_layer, min_size = 40):
        self.viewer = viewer
        self.gt = gt
        self.res = res
        self.annotation_points = []
        self.annotation_order = 0
        self.gt_layer = gt_layer
        self.max_id = np.max(gt)
        self.min_size = min_size



        # - Adding actions
        self.viewer.actions.add("add_annotation", self.add_annotation)
        self.viewer.actions.add("merge_segments", self.merge_segments)
        self.viewer.actions.add("split_segments", self.split_segments)

        # - Binding keys
        with self.viewer.config_state.txn() as s:
            s.input_event_bindings.viewer['shift+keyq'] = "add_annotation"
            s.status_messages['add annotation info'] = "press shift+keyq to select segmentation"
            s.input_event_bindings.viewer['control+shift+keym'] = "merge_segments"
            s.input_event_bindings.viewer['control+shift+keys'] = 'split_segments'
            s.status_messages['merge info'] = "press control+shift+m to merge segmentation"
            s.status_messages['split info'] = "press control+shift+s to split segmentation"
    
    def add_annotation(self, s):
        coordinates = s.mouse_voxel_coordinates
        segment_id = s.selected_values.get("gt").value

        with self.viewer.config_state.txn() as st:
            st.status_messages['annotation info'] = ('Selected segmentation: mouse position = %r selected value = %s' %
                                                (s.mouse_voxel_coordinates, s.selected_values.get("gt").value))

        self.annotation_points.append((coordinates, segment_id))
        print(f"Added annotation at {coordinates} with segment ID {segment_id}")

        if len(self.annotation_points) > 2:
            with self.viewer.config_state.txn() as st:
                st.status_messages['annotation info'] = (f'Too much annotation points. It will automatically delete all points.')
            print(f'Too much annotation points. It will automatically delete all points.')
            self.annotation_points = []

    def merge_segments(self, s):
        if len(self.annotation_points) == 2:
            with self.viewer.config_state.txn() as st:
                st.status_messages['merge info'] = ('Start merging')
            segments = set(annotation[1] for annotation in self.annotation_points)

            segment_ids = list(segments)
            target_segment_id = segment_ids[0]
            merge_segment_id = segment_ids[1]

            print(f"current merge segment id: {merge_segment_id}")
            with self.viewer.config_state.txn() as st:
                st.status_messages['merge info'] = f"current merge segment id: {merge_segment_id}"
            self.gt[self.gt == merge_segment_id] = target_segment_id
            self.gt_layer.invalidate()
            self.annotation_points = []
            with self.viewer.config_state.txn() as st:
                st.status_messages['merge info'] = ('Merging done!')
            print('Merging done!')


    def split_segments(self, s):
        if len(self.annotation_points) != 2:
            print("Error: Exactly two annotation points are required to perform a split. ")
            return 
        print(f"check coordinate: {self.annotation_points[0][0]}")
        if self.annotation_points[0][1] == self.annotation_points[1][1]:
            with self.viewer.config_state.txn() as st:
                st.status_messages['split info'] = f"Start splitting"
            segment_id = self.annotation_points[0][1]
            with self.viewer.config_state.txn() as st:
                st.status_messages['split info'] = f"current split segment id: {segment_id}"
            print(f"current split segment id: {segment_id}")
            mask = self.gt == segment_id
            
            # - check mouse id
            mouse_coord_1 = self.annotation_points[0][0]
            mouse_coord_2 = self.annotation_points[1][0]
            mouse_coord_2d_1 = np.round(mouse_coord_1[1:])
            mouse_coord_2d_2 = np.round(mouse_coord_2[1:])
            target_z = np.round(mouse_coord_1[0])

            non_zero_slices = []
            z_list = []
            for i in range(mask.shape[0]):

                slice_2d = mask[i, :, :]

                if np.any(slice_2d > 0):
                    z_list.append(i)
                    non_zero_slices.append(slice_2d)

            # - split instance slice by slice
            # - set close to the first mouse coordiante, the segment id doesn't need to change
            for non_zero_slice, z_id in zip(non_zero_slices, z_list):
                # - filter out tiny points
                non_zero_slice = non_zero_slice.astype(np.uint8)
                nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(non_zero_slice, connectivity=8)
                filtered_slice = np.zeros_like(non_zero_slice)
                for label_id in range(1, nlabels):
                    area = stats[label_id, cv2.CC_STAT_AREA]
                    if area >= self.min_size:
                        filtered_slice[labels == label_id] = 1

                labeled_array, num_features = label(filtered_slice, structure=generate_binary_structure(2, 2))
                object_slices = find_objects(labeled_array)

                if num_features > 2:
                    print("there are more than two splitting items.")
                    return
                else:
                    reshape_labeled_array = labeled_array[np.newaxis, :, :]
                    if num_features == 1:
                        center_coordinate_1 = center_of_mass(labeled_array == 1)
                        distance_1 = euclidean(center_coordinate_1, mouse_coord_2d_1)
                        distance_2 = euclidean(center_coordinate_1, mouse_coord_2d_2)

                        if distance_1 > distance_2:
                            y_start, y_stop = object_slices[0][0].start, object_slices[0][0].stop
                            x_start, x_stop = object_slices[0][1].start, object_slices[0][1].stop
                            self.gt[z_id, y_start:y_stop, x_start:x_stop][reshape_labeled_array[0, y_start:y_stop, x_start:x_stop] == 1 ] = self.max_id + 1

                    elif num_features == 2:
                        center_coordinate_1 = center_of_mass(labeled_array == 1)
                        center_cooridnate_2 = center_of_mass(labeled_array == 2)
                        distance_mouse_1 = euclidean(center_coordinate_1, mouse_coord_2d_1)
                        distance_mouse_2 = euclidean(center_cooridnate_2, mouse_coord_2d_1)

                        if distance_mouse_1 > distance_mouse_2:
                            y_start, y_stop = object_slices[0][0].start, object_slices[0][0].stop
                            x_start, x_stop = object_slices[0][1].start, object_slices[0][1].stop
                            self.gt[z_id, y_start:y_stop, x_start:x_stop][reshape_labeled_array[0, y_start:y_stop, x_start:x_stop] == 1] = self.max_id + 1
                        else:
                            y_start, y_stop = object_slices[1][0].start, object_slices[1][0].stop
                            x_start, x_stop = object_slices[1][1].start, object_slices[1][1].stop
                            self.gt[z_id, y_start:y_stop, x_start:x_stop][reshape_labeled_array[0, y_start:y_stop, x_start:x_stop] == 2] = self.max_id + 1            
        else:
            print("Error: Exactly two annotation points with the same segment id are required to perform a split. ")
            return   

        self.max_id += 1
        self.gt_layer.invalidate()
        self.annotation_points = []
        with self.viewer.config_state.txn() as st:
            st.status_messages['split info'] = f"Splitting done!"
        print("Splitting done!")

In [12]:
# # - Test 3D split function
# max_id = 10000
# gt_copy = gt.copy()
# test_id1 = 2359
# test_id2 = 2387
# mouse_coord_1 = [4.5000000e+00, 7.3205005e+03, 7.5179995e+03]
# mouse_coord_2 = [4.5000000e+00, 9.3205005e+03, 7.5179995e+03]
# mouse_coord_1 = np.round(mouse_coord_1[1:])
# mouse_coord_2 = np.round(mouse_coord_2[1:])
# gt_copy[gt_copy == test_id1] = test_id2
# mask = gt_copy == test_id2
# min_area = 40


# non_zero_slices = []
# z_list = []
# for i in range(mask.shape[0]):

#     slice_2d = mask[i, :, :]

#     if np.any(slice_2d > 0):
#         z_list.append(i)
#         non_zero_slices.append(slice_2d)

# # - split instance slice by slice
# # - set close to the first mouse coordiante, the segment id doesn't need to change
# count = 0
# for non_zero_slice, z_id in zip(non_zero_slices, z_list):
#     non_zero_slice = non_zero_slice.astype(np.uint8)
#     nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(non_zero_slice, connectivity=8)
    
    
#     filtered_slice = np.zeros_like(non_zero_slice)
#     for label_id in range(1, nlabels):
#         area = stats[label_id, cv2.CC_STAT_AREA]
#         if area >= min_area:
#             filtered_slice[labels == label_id] = 1
#     labeled_array, num_features = label(non_zero_slice, structure=generate_binary_structure(2, 2))
#     filter_labeled_array, filter_num_features = label(filtered_slice, structure=generate_binary_structure(2, 2))
#     object_slices = find_objects(labeled_array)

#     print(count)
#     print(f"the number of labels: {num_features}")
#     print(f"cv2 number of labels: {nlabels}")
#     print(f"the number of filtered labels: {filter_num_features}")


#     if num_features > 2:
#         print("there are more than two splitting items.")
#     else:
#         reshape_labeled_array = labeled_array[np.newaxis, :, :]
#         if num_features == 1:
#             center_coordinate_1 = center_of_mass(labeled_array == 1)
#             distance_1 = euclidean(center_coordinate_1, mouse_coord_1)
#             distance_2 = euclidean(center_coordinate_1, mouse_coord_2)

#             if distance_1 > distance_2:
#                 y_start, y_stop = object_slices[0][0].start, object_slices[0][0].stop
#                 x_start, x_stop = object_slices[0][1].start, object_slices[0][1].stop
#                 gt_copy[z_id, y_start:y_stop, x_start:x_stop][reshape_labeled_array[0, y_start:y_stop, x_start:x_stop] > 0 ] = max_id + 1

#         elif num_features == 2:
#             center_coordinate_1 = center_of_mass(labeled_array == 1)
#             center_cooridnate_2 = center_of_mass(labeled_array == 2)
#             distance_mouse_1 = euclidean(center_coordinate_1, mouse_coord_1)
#             distance_mouse_2 = euclidean(center_cooridnate_2, mouse_coord_1)

#             if distance_mouse_1 > distance_mouse_2:
#                 y_start, y_stop = object_slices[0][0].start, object_slices[0][0].stop
#                 x_start, x_stop = object_slices[0][1].start, object_slices[0][1].stop
#                 gt_copy[z_id, y_start:y_stop, x_start:x_stop][reshape_labeled_array[0, y_start:y_stop, x_start:x_stop] > 0] = max_id + 1
#             else:
#                 y_start, y_stop = object_slices[1][0].start, object_slices[1][0].stop
#                 x_start, x_stop = object_slices[1][1].start, object_slices[1][1].stop
#                 gt_copy[z_id, y_start:y_stop, x_start:x_stop][reshape_labeled_array[0, y_start:y_stop, x_start:x_stop] > 0] = max_id + 1
#     count += 1

# max_id += 1

In [13]:
merge_split_function(viewer, gt, res, gt_layer)
webbrowser.open_new_tab(str(viewer))

True

In [14]:
# # - add local annotation layer
# with viewer.txn() as s:
#     s.layers['annotations'] = neuroglancer.AnnotationLayer()
#     annotations = s.layers['annotations'].annotations
#     s.selected_layer.layer = "annotations"

In [15]:
# # - Add an annotation layer
# import neuroglancer.static_file_server
# import neuroglancer.write_annotations
# tempdir = tempfile.mkdtemp()
# server = neuroglancer.static_file_server.StaticFileServer(
#         static_dir=tempdir, bind_address=ip or "127.0.0.1", daemon=True
#     )
# with viewer.txn() as s:
#     s.layers["annotations"] = neuroglancer.AnnotationLayer(
#         source=f"precomputed://{server.url}",
#         tab="rendering",
#         shader="""
# void main() {
#   setColor(prop_point_color());
#   setPointMarkerSize(prop_size());
# }
# """,
#     )
#     s.selected_layer.layer = "annotations"
#     s.selected_layer.visible = True
#     s.show_slices = False


In [16]:
# def write_some_annotations(
#     output_dir: str, coordinate_space: neuroglancer.CoordinateSpace, target_coordinate
# ):
#     writer = neuroglancer.write_annotations.AnnotationWriter(
#         coordinate_space=coordinate_space,
#         annotation_type="point",
#         properties=[
#             neuroglancer.AnnotationPropertySpec(id="size", type="float32"),
#             neuroglancer.AnnotationPropertySpec(id="cell_type", type="uint16"),
#             neuroglancer.AnnotationPropertySpec(id="point_color", type="rgba"),
#         ],
#     )

#     writer.add_point(target_coordinate, size=50, cell_type=16, point_color=(255, 255, 255, 255))
#     writer.write(output_dir)

In [17]:
# write_some_annotations(output_dir=tempdir, coordinate_space=res, target_coordinate=[0, gt.shape[1]/2, gt.shape[2]/2])

In [18]:
# import neuroglancer.random_token


# class Annotator:
#     def __init__(self, viewer, gt, res):
#         self.viewer = viewer
#         self.gt = gt
#         self.res = res
#         self.annotation_points = []
#         self.annotation_order = 0

#         # - Adding actions
#         self.viewer.actions.add("add-annotation", self.add_annotation)
#         # self.viewer.actions.add("merge-segments", self.merge_segments)
#         # self.viewer.actions.add("clear-annotations", self.clear_annotations)

#         # - Binding keys
#         with self.viewer.config_state.txn() as s:
#             s.input_event_bindings.viewer['control+shift+mousedown0'] = "add-annotation"
#             s.input_event_bindings.viewer['control+shift+keym'] = "merge-segments"
#             s.input_event_bindings.viewer['control+keyd'] = "clear-annotations"
    
#     def add_annotation(self, s):
#         coordinates = s.mouse_voxel_coordinates
#         segment_id = s.selected_values.get("gt")

#         if segment_id is None:
#             print(f"No segment found at {coordinates}")
#             return
#         with self.viewer.txn() as state:
#             pt = neuroglancer.PointAnnotation(point = coordinates, id = f'point{self.annotation_order}')
#             annotations.append(pt)
#         self.annotation_points.append((coordinates, segment_id))
#         self.annotation_order += 1
#         print(f"Added annotation at {coordinates} with segment ID {segment_id}")


In [19]:
# # - Test add-annotation function
# annotator = Annotator(viewer, gt, res)


In [20]:
# import numpy as np

# with viewer.txn() as s:
#     s.layers['points'] = neuroglancer.LocalAnnotationLayer(dimensions=res)

# num_actions = 0
# def logger(s):
#     global num_actions
#     num_actions += 1
#     with viewer.config_state.txn() as st:
#         st.status_messages['hello'] = ('Got action %d: mouse position = %r' %
#                                     (num_actions, s.mouse_voxel_coordinates))

#     print('Log event')
#     print('Mouse position: ', np.array(s.mouse_voxel_coordinates))
#     print('Layer selected values:', (np.array(list(viewer.state.layers['segmentation'].segments))))
#     with viewer.txn() as s:
#         point = np.array(s.mouse_voxel_coordinates)
#         point_anno = neuroglancer.PointAnnotation(
#                          id=repr(point),
#                          point=point)
#         s.layers['points'].annotations = [point_anno]


# viewer.actions.add('logger', logger)
# with viewer.config_state.txn() as s:
#     s.input_event_bindings.viewer['keyl'] = 'logger'
#     s.status_messages['hello'] = 'Add a promt for neuroglancer'

### Swap RAM to Disk

In [21]:
# # Helper functions
# def choose_first_n_images_disk(file_path, dataset_name, n, output_array):
#     with h5py.File(file_path, 'r') as fl:
#         tmp_dataset = fl[dataset_name]
#         for i in range(n):
#             output_array[i] = tmp_dataset[i]

# gt_file = 'gt_array.dat'
# shape = (100, 15260, 15217)
# gt_array = np.memmap(gt_file, dtype = np.uint32, mode = 'w+', shape=shape)
# choose_first_n_images_disk(seg_output_file, 'seg_images', 100, gt_array)
# gt_array.flush()


In [22]:
# # - Get unique ids
# chunk_size = 1
# for i in range(0, gt_array.shape[0], chunk_size):
#     chunk = gt_array[i:i+chunk_size, :, :]
#     unique_chunk = np.unique(chunk).astype('unint32')

In [23]:

# im_array = choose_first_n_images(raw_output_file, 'raw_images', 100, array_dtype = None)

In [24]:
# ip = 'localhost'
# port = 9999
# neuroglancer.set_server_bind_address(bind_address=ip, bind_port=port)
# viewer = neuroglancer.Viewer()

# res = neuroglancer.CoordinateSpace(
#     names=['z', 'y', 'x'],
#     units=['nm', 'nm', 'nm'],
#     scales=[60, 4, 4]
# )

# print('Load raw image and segmentation.')


# def ngLayer(data, res, oo=[0, 0, 0], tt='segmentation'):
#     return neuroglancer.LocalVolume(data, dimensions=res, volume_type=tt, voxel_offset=oo)
# im_layer = ngLayer(im_array, res, tt='image')
# gt_layer = ngLayer(gt_array, res, tt='segmentation')
# with viewer.txn() as s:
#     s.layers.append(name='im', layer=im_layer)
#     s.layers.append(name='gt', layer=gt_layer)


# print(viewer)

In [25]:
# with viewer.txn() as s:
#     s.layers['segmentation'].segments.add(204336)
#     s.layers['segmentation'].segments.add(213086)